#  Intégration continue avec Python

## Qu'est-ce que l'intégration continue ?

L'intégration continue est une pratique consistant, de manière automatique,
à fréquemment tester les effets d'une modification faite à un code ou à un
document faisant parti d'un projet informatique.

Cette pratique permet ainsi de détecter de manière précoce des possibilités
de *bug* ou l'introduction d'un changement non anticipé. Tout comme `Git`,
cette pratique devient un standard dans les domaines collaboratifs.

L'intégration continue permet de sécuriser le travail, puisqu'elle offre un
filet de sécurité (par exemple un test sur une machine à la configuration
arbitraire), mais permet aussi de déployer en temps réel certaines
évolutions. On parle parfois de déploiement en continu, complémentaire de
l'intégration continue. Cette approche réduit ainsi
la muraille de Chine entre un
analyste de données et une équipe de développeurs d'application. Elle offre donc
plus de contrôle, pour le producteur d'une analyse statistique, sur la
valorisation de celle-ci.

L'intégration continue fonctionne très bien sur `Gitlab` et sur `Github`.
A chaque interaction avec le dépôt distant (`push`), une série d'instruction
définie par l'utilisateur est exécutée. `Python` et `R` s'intègrent très bien avec l'intégration continue grâce
à un certain nombre d'images de base (concept sur lequel nous allons revenir)
qui peuvent être customisées pour répondre à une certaine configuration
nécessaire pour exécuter des codes
([voir ici pour quelques éléments sur R](https://linogaliana.gitlab.io/collaboratif/package.html#utiliser-lint%C3%A9gration-continue-de-gitlab).
C'est une méthode idéale pour améliorer la reproductibilité d'un projet: les
instructions exécutées le sont dans un environnement isolé et contrôlé, ce qui
diffère d'une machine personnelle.

## Comment fonctionne l'intégration continue ?

L'intégration continue repose sur le système de la *dockerisation* ou *conteneurisation*.
La technologie sous jacente s'appelle `Docker`.
Il s'agit d'une technologie qui permet la construction
de machines autosuffisantes
(que l'on nomme **containeurs**) répliquant un environnement
contrôlé (que l'on nomme **image**).

On parle de *pipelines* pour désigner une suite de tâches pour partir de 0
(généralement une machine `Linux` à la configuration minimale) et aboutir
à l'issue d'une série d'instructions définies par l'utilisateur.

L'objectif est de trouver une image la plus
parcimonieuse possible, c'est-à-dire à la configuration minimale, qui permet
de tester de faire tourner le code voulu. Dans le domaine de la *datascience*,
les images de [JupyterHub](https://hub.docker.com/r/jupyterhub/jupyterhub/) constituent
un bon point de départ. Il est également très simple de construire son image
de rien, ce qui sera proposé par la suite.

Quand on utilise un dépôt `Github` ou `Gitlab`, des services automatiques
d'intégration continue peuvent être utilisés:

* `Gitlab CI`: solution pleinement intégrée à un dépôt Gitlab. Très généraliste
et permettant des *pipelines* très complexes
([voir l'intégration continue du projet utilitR, une documentation pour R](https://gitlab.com/linogaliana/documentationR/-/blob/master/.gitlab-ci.yml)).
Il est également possible de
l'utiliser avec un dépôt stocké sur `Github`. L'inconvénient de cette approche
est qu'elle est assez lente.
* `Github Actions`: c'est l'alternative (relativement récente) au service d'intégration continue de
Gitlab uniquement basée sur les technologies Github. La très forte
dynamique de développement a rendu ce service incontournable.

Historiquement, il existait d'autres services d'intégration continue, notamment
`Travis CI` ou `AppVeyor`[^1]

[^1]: Ces services d'intégration continue étaient utilisés lorsque `Github`
ne proposait pas encore de service intégré, comme le faisait `Gitlab`.
Ils sont de moins en moins fréquemment utilisés.

## Fonctionnement des actions Github

Les actions `Github` fonctionnent par couches successives au sein desquelles
on effectue un certain nombre d'instructions. 
La meilleure manière d'apprendre les actions `Github` est, certes, de [lire la
documentation officielle](https://docs.github.com/en/actions) mais surtout,
à mon avis, de regarder quelques *pipelines* pour comprendre la démarche. 

L'un des intérêts des Github Actions est la possibilité d'avoir un *pipeline*
proposant une intrication de langages différents pour avoir une chaine de 
production qui propose les outils les plus efficaces pour répondre à un
objectif en limitant les verrous techniques. 

Par exemple, le *pipeline* de ce cours, disponible
sur `Github`
<a href="https://github.com/linogaliana/python-datascientist/blob/master/.github/workflows/prod.yml" class="github"><i class="fab fa-github"></i></a>
propose une intrication des langages
`Python` et `R` avec des technologies `Anaconda` (pour contrôler
l'environnement `Python` comme expliqué dans les chapitres précédents)
et `Javascript` (pour le déploiement d'un site web avec le service tiers
`Netlify`)[^2]. Cette chaîne de production multi-langage permet que
les mêmes fichiers sources génèrent un site web et des notebooks disponibles
sur plusieurs environnements.

[^2]: Pour réduire le temps nécessaire pour construire le site *web*, ce
*pipeline* s'appuie sur un environnement `docker` construit sur un autre dépôt
disponible également sur `Github`
<a href="https://github.com/linogaliana/python-datascientist-docker/blob/master/.github/workflows/prod.yml" class="github"><i class="fab fa-github"></i></a>.
Celui-ci part d'une configuration système `Linux` et construit un environnement
`Anaconda` à partir d'un fichier `environment.yml` qui liste toutes les dépendances
nécessaires pour exécuter les morceaux de code du site *web*. 
Cet environnement `Anaconda` est construit grâce à l'outil `mamba` qui permet
d'aller beaucoup plus vite dans la constitution d'environnements que ne le
permet `conda`. 

Les couches qui constituent les étapes du *pipeline*
portent ainsi le nom de `steps`. Un *step* peut comporter un certain 
nombre d'instructions ou exécuter des instructions pré-définies.
L'une de ces instructions prédéfinies est, par exemple,
l'[installation de Python](https://github.com/actions/setup-python)
ou l'[initialisation d'un environnement conda](https://github.com/marketplace/actions/setup-miniconda). 
La documentation officielle de `Github` propose un
[fichier qui peut servir de modèle](https://docs.github.com/en/actions/automating-builds-and-tests/building-and-testing-nodejs-or-python?langId=py)
pour tester un script `Python` voire l'uploader de manière automatique
sur `Pypi`.

## Intégration continue avec `Python`: tester un notebook

Cette section n'est absolument pas exaustive. Au contraire, elle ne fournit
qu'un exemple minimal pour expliquer la logique de l'intégration continue. Il
ne s'agit ainsi pas d'une garantie absolue de reproductibilité d'un *notebook*.

`Github` propose une action officielle pour utiliser `Python` dans un
*pipeline* d'intégration continue. Elle est disponible sur le
[MarketPlace Github](https://github.com/marketplace/actions/setup-python).
Il s'agit d'un bon point de départ, à enrichir.

Le fichier qui contrôle les instructions exécutées dans l'environnement `Actions`
doit se trouver dans le dossier `.github/workflows/`
(:warning: ne pas oublier le point au début du
nom du dossier). Il doit être au format `YAML` avec une extension `.yml`.
Il peut avoir n'importe quel nom mais mieux vaut lui donner un nom signifiant,
par exemple `prod.yml` pour un fichier contrôlant une chaîne de production. 

### Lister les dépendances

Avant d'écrire les instructions à exécuter par `Github`, il faut définir un
environnement d'exécution car `Github` ne connaît pas la configuration `Python`
dont vous avez besoin.

Il convient ainsi de lister les dépendances nécessaires dans un fichier
`requirements.txt`, comme expliqué dans la partie
[Bonnes pratiques](#bonnespratiques), ou un fichier `environment.yml`.
Ce fichier fait la liste des dépendances à installer.
Si on fait le choix de l'option `environment.yml`,
le fichier prendra ainsi la forme
suivante:

```yaml
channels:
  - conda-forge

dependencies:
  - python
  - jupyter
  - jupytext
  - matplotlib
  - nbconvert
  - numpy
  - pandas
  - scipy
  - seaborn
```

Le choix du *channel* `conda-forge` vise à contrôler le dépôt utilisé par
`Anaconda`.

Ne pas oublier de mettre ce fichier sous contrôle de version et de l'envoyer
sur le dépôt par un `push`.

### Tester un notebook `myfile.ipynb`

Dans cette partie, on va supposer que le *notebook* à tester s'appelle `myfile.ipynb`
et se trouve à la racine du dépôt.

Le modèle suivant, expliqué en dessous, fournit un modèle de recette pour
tester un notebook:

TO BE COMPLETED

## Annexe: la même approche avec `Travis`

```shell
# Modèle de fichier .travis.yml
language: python
python:
  - "3.7"

install:
  - sudo apt-get update
  # We do this conditionally because it saves us some downloading if the
  # version is the same.
  - if [[ "$TRAVIS_PYTHON_VERSION" == "2.7" ]]; then
      wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh -O miniconda.sh;
    else
      wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh;
    fi
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a
  - conda env create -n test-environment python=$TRAVIS_PYTHON_VERSION -f environment.yml
  - source activate test-environment

script:
  - jupytext --to py --execute myfile.ipynb
```

### Explications

Les lignes:

```shell
language: python
python:
  - "3.7"
```

définissent la version de Python qui sera utilisée. Cependant, il convient
d'installer `Anaconda` (en fait une version minimaliste d'`Anaconda` nommée
`Miniconda`) ainsi que configurer la machine pour utiliser Anaconda plutôt
que la version de base de `Python`. Ce sont les lignes suivantes
qui contrôlent ces opérations:

```yaml
install:
  - sudo apt-get update
  # We do this conditionally because it saves us some downloading if the
  # version is the same.
  - if [[ "$TRAVIS_PYTHON_VERSION" == "2.7" ]]; then
      wget https://repo.continuum.io/miniconda/Miniconda2-latest-Linux-x86_64.sh -O miniconda.sh;
    else
      wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh;
    fi
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a
```

Enfin, le reste de la tâche `install` est consacrée à la construction d'un
environnement Anaconda cohérent avec les packages définis dans `environment.yml`:

```yaml
- conda env create -n test-environment python=$TRAVIS_PYTHON_VERSION -f environment.yml
- source activate test-environment
```

Tout cela permet de construire un conteneur qui a vocation à être suffisant
pour exécuter `myfile.ipynb`. C'est l'objet de la tâche `script`:

```yaml
script:
  - jupytext --to py --execute myfile.ipynb
```

`jupytext` est une extension de `jupyter` qui fournit des éléments pour passer d'un
notebook à un autre format. En l'occurrence, il s'agit de convertir
un *notebook* en
script `.py` et l'exécuter. Ce test pourrait également être fait en n'utilisant
que `Jupyter`:

```yaml
script:
  - jupyter nbconvert --to notebook --execute --inplace myfile.ipynb
```